In [51]:
from typing import List, Dict, Any, Union
import os

In [52]:
data = os.path.abspath("data/german_credit_data_biased_training.csv")

## Gather dataset column information

In [53]:
from pathlib import Path
import pandas as pd
import numpy as np
import json

def load_data(data_path: Union[str, Path]):
    df = pd.read_csv(data_path)

    for col in df.columns:
        if df[col].dtype == np.dtype("O"):
            df[col] = df[col].astype("category")

    column_map: Dict[str, Any] = {}
    column_map["columns"] = list(df.columns)
    column_map["label_columns"] = {}
    column_map["column_types"] = {}
    
    for col in column_map["columns"]:
        if isinstance(df[col].dtype, pd.CategoricalDtype):
            column_map["label_columns"][col] = list(df[col].dtype.categories)
        
        dtype_str = str(df[col].dtype)
        if dtype_str != "category":
            if dtype_str not in column_map["column_types"]:
                column_map["column_types"][dtype_str] = []
            column_map["column_types"][dtype_str].append(col)
    return df, column_map


In [54]:
g_df, german_data = load_data(data)
german_data

{'columns': ['CheckingStatus',
  'LoanDuration',
  'CreditHistory',
  'LoanPurpose',
  'LoanAmount',
  'ExistingSavings',
  'EmploymentDuration',
  'InstallmentPercent',
  'Sex',
  'OthersOnLoan',
  'CurrentResidenceDuration',
  'OwnsProperty',
  'Age',
  'InstallmentPlans',
  'Housing',
  'ExistingCreditsCount',
  'Job',
  'Dependents',
  'Telephone',
  'ForeignWorker',
  'Risk'],
 'label_columns': {'CheckingStatus': ['0_to_200',
   'greater_200',
   'less_0',
   'no_checking'],
  'CreditHistory': ['all_credits_paid_back',
   'credits_paid_to_date',
   'no_credits',
   'outstanding_credit',
   'prior_payments_delayed'],
  'LoanPurpose': ['appliances',
   'business',
   'car_new',
   'car_used',
   'education',
   'furniture',
   'other',
   'radio_tv',
   'repairs',
   'retraining',
   'vacation'],
  'ExistingSavings': ['100_to_500',
   '500_to_1000',
   'greater_1000',
   'less_100',
   'unknown'],
  'EmploymentDuration': ['1_to_4',
   '4_to_7',
   'greater_7',
   'less_1',
   'unemp

In [55]:
r_df, random_data = load_data("data/random_data.csv")
random_data

{'columns': ['Category', 'Value1', 'Value2'],
 'label_columns': {'Category': ['A', 'B', 'C']},
 'column_types': {'int64': ['Value1'], 'float64': ['Value2']}}

In [101]:
from mapepire_python import Connection, connect


def get_mapepire_conn(config: Union[str, Path], **kwargs) -> Connection:
    return connect(config, **kwargs)



db2 = get_mapepire_conn("connect.ini")
db2

## Test Connection with simple fetch

In [74]:
cur = db2.execute('select * from sample.employee')
cur.fetchone()

{'id': 'fetchMore4',
 'data': [{'EMPNO': '000010',
   'FIRSTNME': 'CHRISTINE',
   'MIDINIT': 'I',
   'LASTNAME': 'HAAS',
   'WORKDEPT': 'A00',
   'PHONENO': '3978',
   'HIREDATE': '01/01/65',
   'JOB': 'PRES',
   'EDLEVEL': 18,
   'SEX': 'F',
   'BIRTHDATE': None,
   'SALARY': 158250.0,
   'BONUS': 4000.0,
   'COMM': 4220.0}],
 'is_done': False,
 'success': True}

## Utils

In [102]:
from mapepire_python.client import SQLJob

sql_types = {
    'int64': 'INTEGER',
    'float64': 'DOUBLE',
    'object': 'VARCHAR',
    'bool': 'BOOLEAN',
    'datetime64[ns]': 'TIMESTAMP',
    'timedelta[ns]': 'INTERVAL',
    'category': 'VARCHAR'
}

def table_exists(table_name: str, conn: Connection) -> Union[dict, bool]:
    pass

def build_check_constraints_sql(
    table_name: str, column_info: Dict[str, Any]
) -> List[str]:
    constraints = []
    for label_col, levels in column_info["label_columns"].items():
        str_levels = ["'" + level.replace("'", "''") + "'" for level in levels]
        constraints.append(f"CHECK ({label_col} IN ({','.join(str_levels)}))")
    return constraints

def build_create_table_sql(
    name: str, column_info: Dict[str, Any], identity_start: int = 1
) -> str:
    sql_safe_table_name = name

    cols_sql = []
    for col in column_info["label_columns"]:
        cols_sql.append(f"{col} VARCHAR(32000) NOT NULL")
    
    for col_type, cols in column_info["column_types"].items():
        for col in cols:
            cols_sql.append(f" {col} {sql_types[col_type]} NOT NULL")
    return (
        f"CREATE or REPLACE TABLE {sql_safe_table_name} ("
        + f"ACCOUNT_ID BIGINT GENERATED BY DEFAULT AS IDENTITY (START WITH {identity_start}),"
        + ", ".join(cols_sql)
        + ", "
        + ", ".join(build_check_constraints_sql(name, column_info))
        + ")"
    )
    
    
def df_to_sql(
    name: str,
    conn: Connection,
    df: pd.DataFrame,
    columns: Dict[str, Any],
    identity_start: int = 1,
) -> None:
    sql_safe_name = name.replace('"', "")
    sql = build_create_table_sql(sql_safe_name, columns, identity_start)

    iStmtColsSql = ", ".join([f'{col}' for col in columns["columns"]])
    iValues = ",".join(["?" for _ in range(len(columns["columns"]))])
    iSql = f'INSERT INTO {sql_safe_name} ({iStmtColsSql}) VALUES({iValues})'
    df_values = list([list(x) for x in df.loc[:, columns["columns"]].values])
    
    print(sql)
    print(iSql)
    print(df_values)
    
    conn.execute(sql)
    conn.executemany(iSql, df_values)
    

In [103]:
df_to_sql("sample.randomdata", db2, r_df, random_data)

CREATE or REPLACE TABLE sample.randomdata (ACCOUNT_ID BIGINT GENERATED BY DEFAULT AS IDENTITY (START WITH 1),Category VARCHAR(32000) NOT NULL,  Value1 INTEGER NOT NULL,  Value2 DOUBLE NOT NULL, CHECK (Category IN ('A','B','C')))
INSERT INTO sample.randomdata (Category, Value1, Value2) VALUES(?,?,?)
[['A', 61, 16.685430556951093], ['A', 24, 7.143340896097039], ['A', 81, 32.54442364744264], ['C', 70, 2.820578951355013], ['C', 30, 36.09993861334124], ['C', 92, 46.927635450787506], ['B', 96, 0.0389382920507164], ['C', 84, 49.610577964560875], ['B', 84, 30.874075481385827], ['B', 97, 30.582658024414044]]


In [104]:
cur = db2.execute("select * from sample.randomdata")
cur.fetchall()

{'id': 'fetchMore6',
 'data': [{'ACCOUNT_ID': 1,
   'CATEGORY': 'A',
   'VALUE1': 61,
   'VALUE2': 16.685430556951093},
  {'ACCOUNT_ID': 2,
   'CATEGORY': 'A',
   'VALUE1': 24,
   'VALUE2': 7.143340896097039},
  {'ACCOUNT_ID': 3,
   'CATEGORY': 'A',
   'VALUE1': 81,
   'VALUE2': 32.54442364744264},
  {'ACCOUNT_ID': 4,
   'CATEGORY': 'C',
   'VALUE1': 70,
   'VALUE2': 2.820578951355013},
  {'ACCOUNT_ID': 5,
   'CATEGORY': 'C',
   'VALUE1': 30,
   'VALUE2': 36.09993861334124},
  {'ACCOUNT_ID': 6,
   'CATEGORY': 'C',
   'VALUE1': 92,
   'VALUE2': 46.927635450787506},
  {'ACCOUNT_ID': 7,
   'CATEGORY': 'B',
   'VALUE1': 96,
   'VALUE2': 0.0389382920507164},
  {'ACCOUNT_ID': 8,
   'CATEGORY': 'C',
   'VALUE1': 84,
   'VALUE2': 49.610577964560875},
  {'ACCOUNT_ID': 9,
   'CATEGORY': 'B',
   'VALUE1': 84,
   'VALUE2': 30.874075481385827},
  {'ACCOUNT_ID': 10,
   'CATEGORY': 'B',
   'VALUE1': 97,
   'VALUE2': 30.582658024414044},
  {'ACCOUNT_ID': 11,
   'CATEGORY': 'A',
   'VALUE1': 61,
   'VA